<a href="https://colab.research.google.com/github/MelihKlc/Deep-Learning/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP with TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences or text)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

# Token dediğimiz şey ya bir word ya da character oluyor. Bizim seçimimize göre

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-08-18 14:22:43--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-08-18 14:22:43 (59.2 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback , plot_loss_curves , compare_historys , unzip_data , walk_through_dir , calculate_results

# Get the text data

The dataset we are going to use it NLP dataset for text samples tweets labelled as disaster or not disaster

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-08-18 14:22:46--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.119.128, 108.177.121.128, 142.250.1.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-08-18 14:22:47 (115 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

# Become one with the data

In [5]:
# Visualizing our text dataset

import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# If target label has 1 , it means that our sentence is about disasters

train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training data because of we dont want to be overfitting

train_df_shuffled = train_df.sample(frac = 1 , random_state = 42) # Frac = 1 dememizin sebebi bütün datamızı shuffle etmek istiyoruz.


In [8]:
# What does the our test_dataset look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class?

train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples?

len(train_df) , len(test_df)

(7613, 3263)

In [11]:
# Lets visualize some random training examples.
import random
random_index = random.randint( 0 , len(train_df) - 5)  # Create random indexes not higher than the total number of samples

for row in train_df[["text" , "target"]][random_index : random_index + 5].itertuples():  # Bu satırda, train_df veri çerçevesinde rastgele seçilen 5 satırı (veya kalan kadar satır varsa) içeren bir alt veri çerçevesini seçiyoruz. Sadece "text" ve "target" sütunlarını seçiyoruz. itertuples() işlevi, seçilen satırları birer birer dönmek için kullanılır.
  _ , text , target = row
  print(f"Target : {target} " , "(real_disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n {text}")
  print("---\n")


Target : 0  (not real disaster)
Text:
 I can't listen to Darude Sandstorm without expecting airhorns now
---

Target : 0  (not real disaster)
Text:
 Now playing: Darude - Sandstorm - radio edit http://t.co/DUdAIrBBPo http://t.co/padosfyXnM
---

Target : 1  (real_disaster)
Text:
 Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/bgM4cSrbVd
---

Target : 0  (not real disaster)
Text:
 I slammed my phone to the ground and then screamed ahahahga
---

Target : 0  (not real disaster)
Text:
 @InfiniteGrace7 I just screamed to the world how much I love My Little Pony ??
---



# Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

train_sentences , val_sentences , train_labels , val_labels = train_test_split(train_df_shuffled["text"].to_numpy() ,train_df_shuffled["target"].to_numpy() , test_size=0.1 , random_state = 42)

In [13]:
len(train_sentences) , len(val_sentences) , len(train_labels) , len(val_labels)

(6851, 762, 6851, 762)

In [14]:
 # Check the first 10 samples

 train_sentences[ : 10] , train_labels[ : 10]  # Train sentence içindeki cümleler train_labels taki outputlarla eşleşiyor

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

When dealing with a text problem , one of the first things you will have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this namely:
* Tokenization - Direct mapping of token (a token could be a word or a character) to number.
* Embedding - Create a matrix of feature vector for each token (the size of the feature vector can be and this embedding can be learn)

# Text vectorization (tokenization , Converting text into numbers)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#ngram substringleri birleştiriyor. Mesela elimizde 'Imagine getting flattened by Kurt Zouma' cümlesi var. ngram = 2 dersek , "Image getting" , "flattened by" , "Kurt Zouma"  gibi ikişer ikişer substringleri birleştiriyor.

# Default values of TextVectorization
text_vectorizer = TextVectorization(max_tokens = 10000 , # How many words in the vocabulary ( automatically add <00V>)
                                                        standardize = "lower_and_strip_punctuation" ,  # Bütün harfleri küçük yapıp noktalama işaretlerini siliyor çünkü bunlar disaster olup olmadıgını anlamamıza yardım eden şeyler değil.
                                                        split = "whitespace"  , # Tokenlar arasında boşluk bırakmak için.
                                                        ngrams = None , # Create a groups of n-word
                                                        output_mode = "int" , # How to map tokens to number
                                                        output_sequence_length = None , # How long do you want your sequences to be. (None atadıgımızda en uzun tokeni alıp bütün tokenlere o lengthi veriyor. Kısa olan sentences ların sonu 0 larla dolduruluyor.)
                                                        pad_to_max_tokens = True
                                    )

In [16]:
train_sentences[0].split()  # Split bu işe yarıyor.

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [17]:
# Find the average number of tokens (words) in the training tweets
# We could just pad them to the longest , but to keep our data small , we will just find the average lenght and then make sure every tweet gets turned into numbers but those list of numbers is the same size as the average throughout every sample.

round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [18]:
# Setup text vectorization variables.

max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # Max length our sequences will be (e.g. How many words from a Tweet does model see?). Mesela tweet 30 kelime uzunlugunda ama biz bunu 15 atadıgımız icin modelimiz ilk 15 kelimesini görecek

# Uptading our textvectorizer

text_vectorizer = TextVectorization(max_tokens = max_vocab_length ,  # Bizim bütün tweetlerimizden 10000 tane alacak bu 10000 most common lardan oluşacak
                                    output_mode = "int" ,
                                    output_sequence_length = max_length
                                    )


In [19]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [20]:
# Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)  # CNN deki fit işlemi gibi artık bütün train sentenceslarımıza text_vectorizerde ne yapılmasını istediysek uygulanacak.

In [21]:
# We are trying to do here is map the word data that we have to numerical form

In [22]:
# Create a sample sentence and tokenize it

sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])   # Text vectorizer ı biz liste şeklinde oluşturdugumuz icin girdigimiz örnekleri [ ] içinde yazıyoruz.
# Burda shape in 0. indexi sequence i veriyor. 1 tam cümle girdiğimiz için sequence 1.
# Shape in 1. indexi de length veriyor. Max olarak 15 dediğimiz için buna adapt oluyo bizim sample ımız.

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose a random sentence from our train_sentences data

import random
random_sentences = random.choice(train_sentences)

print(f"Original sentence : \n {random_sentences} \n\n  Vectorized form of our random sentence: \n")
text_vectorizer([random_sentences])

Original sentence : 
 @SergioPiaggio 'IÛªd worked so hard to get to that level that I wasnÛªt going to let the injury define me. I was going to define it.Û Cool 

  Vectorized form of our random sentence: 



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[8643, 5331, 1733,   28,  892,    5,   52,    5,   16,  813,   16,
           8, 6913,  104,    5]])>

In [24]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()  # Get all of the unique words in our training data
top_5_words = words_in_vocab[ : 5 ] # Get the most common words in our data
bottom_5_words = words_in_vocab[ -5 :  ] # Get the least common words

print(f"Number of words in vocab : {len(words_in_vocab)} \n")
print(f"Most common words : {top_5_words} \n")
print(f"Least common words  : {bottom_5_words} \n")

Number of words in vocab : 10000 

Most common words : ['', '[UNK]', 'the', 'a', 'in'] 

Least common words  : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'] 



# Creating an Embedding using an Embedding Layer

To make our embedding we are going to use TensorFlow embedding layer.

The parameters we care most about for our embedding layers

* 'input_dim' : the size of our vocabulary
* 'output_dim' : the size of the output embedding vector , for example , a value of 100 would mean each token gets represented by a vector 100 long.
* 'input_length' : lenght of the sequences being passed to the embedding layer.

# **Note** : Neural networks 8 e bölünebilen sayılarla çok iyi çalışıyor

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding( input_dim = max_vocab_length ,  # set input shape
                             output_dim = 128 , # Output shape. Every token in the format of 128 long vector
                              input_length = max_length  # How long is each input
                              )

In [26]:
# Get a random sentence from our training data

random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence}\n")
print("Embedded form")
sample_embed = embedding(text_vectorizer([random_sentence]))  # Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed

Original text : 
 Demolition Means Progress: Flint Michigan and the Fate of the American Metropolis Highsmith https://t.co/ZvoBMDxHGP

Embedded form


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02536899,  0.04975064,  0.04531486, ...,  0.04033859,
          0.02942633,  0.01726137],
        [-0.00965609,  0.03112185,  0.01632122, ..., -0.01998364,
         -0.00841166,  0.03210044],
        [-0.01363263, -0.04059486,  0.00960726, ...,  0.0269217 ,
         -0.01254296,  0.04731056],
        ...,
        [-0.01412129, -0.02558997,  0.03156263, ..., -0.02259362,
          0.03602357, -0.01284522],
        [-0.01412129, -0.02558997,  0.03156263, ..., -0.02259362,
          0.03602357, -0.01284522],
        [-0.00050489, -0.03629545,  0.03552065, ...,  0.00640681,
         -0.04900563,  0.04188022]]], dtype=float32)>

In [27]:
# Check out a single token's embedding

sample_embed[0][0] , sample_embed[0][0].shape , sample_embed

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-2.5368989e-02,  4.9750637e-02,  4.5314860e-02,  5.1313639e-03,
         3.6592294e-02,  1.0269992e-03,  3.7733022e-02,  3.0495692e-02,
        -1.0252379e-02,  2.0583678e-02, -3.1429686e-02, -4.2386364e-02,
         1.1711717e-03,  2.5520828e-02, -3.0533804e-02, -4.6086907e-03,
         2.4970416e-02,  3.9121162e-02, -1.4882050e-02, -3.4789313e-02,
        -4.6104420e-02,  2.8650835e-04,  1.4943842e-02,  4.7540776e-03,
         2.1904718e-02,  1.2770090e-02,  4.4552866e-02, -4.8190467e-03,
        -4.6311796e-02, -2.8489470e-02,  3.6288425e-04, -3.0012727e-03,
         4.9475227e-02, -2.3742605e-02, -5.2202493e-05, -3.7650466e-02,
         3.2933664e-02,  2.3520473e-02,  2.4384093e-02, -1.7920494e-02,
         4.8910905e-02,  8.0694556e-03, -7.3417649e-03,  2.0037506e-02,
         1.0574758e-02, -4.7010422e-02,  2.6305329e-02,  4.3807913e-02,
         2.5681440e-02, -4.9889721e-02,  3.6801483e-02, -4.8700668e-02,
         6.8055

# Modelling a text dataset (running a series of experimentation)

Now we have got a way to turn our text sequences into numbers , it is time to start building a series of modelling experiments.

We will start with a baseline and move on from there

* Model 0 : Naive Bayes (baseline)
* Model 1 : Feed-forward neural network
* Model 2 : LSTM model (RNN)
* Model 3 : GRU model (RNN)
* Model 4 : Bidirectional LSTM (RNN)
* Model 5 : 1D Convolutional neural network (CNN)
* Model 6 : TensorFlow Hub pre-trained Feature extractor (using transfer learning for NLP)
* Model 7 : Same as model 6 but with %10 of training data

How are we going to approach all of this ?

Use standart steps in modelling with tensorflow

* Create a model
* Compile o model
* Fit the model
* Evaluate the model

## Model 0 : Getting a baseline

As with all machine learning modelling experiments , it is important to create a baseline model so you have got a benchmark for future experiments to build upon

To create our baseline, We will use sklearn Multinomial naive bayes using the TF-IDF formula to convert our words to numbers.

> **Note:** It is common practice to use non deep learning algorithms as a baseline because of their speed then later using deep learning to see if you can improve upon them

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline(
    [
     ("tfidf" , TfidfVectorizer())   , # Convert words to numbers using tfidf
     ("clf" , MultinomialNB() ) # Model to text

    ]

)

# Fit the pipeline to the training data
model_0.fit(train_sentences , train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our model
baseline_score = model_0.score(val_sentences , val_labels)  # In sklearn we use score instead of evaluate
baseline_score
# Bu bize accuracy veriyor.


0.7926509186351706

In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[ : 20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [31]:
val_labels[ : 20]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

# Creating an evaluation function for our model experiment

We could evaluate all of our model's predictions with different metrics every time , however , this will be cumbersome(hantal) and could easliy be fixed with a function

Lets create one to compare our model's predictions with the truth labels using following metrices

* Accuracy
* Precision
* Recall
* F1-score

In [32]:
# Function to evaluate : accuracy , precision , recall , f1-score
from sklearn.metrics import accuracy_score , precision_recall_fscore_support

def calculate_results(y_true , y_pred):
  """
  Calculates model accuracy , precision , recall and f1 score of a binary classification model
  """

  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true , y_pred)

  # Calculate model precision , recall and f1 score using "weighted" average

  model_precision , model_recall , model_f1 , _ = precision_recall_fscore_support(y_true , y_pred , average = "weighted")
  model_results = {
      "accuracy": model_accuracy ,
      "precision": model_precision ,
      "recall": model_recall ,
      "f1-score": model_f1
  }
  return model_results

In [33]:
# Get model results

baseline_results = calculate_results(y_true = val_labels ,
                                     y_pred = baseline_preds
                                     )
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

# Model 1 : A simple dense model

In [34]:
# Create a tensorboard callback (need to create a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
save_dir = "model_logs"

In [35]:
# Build a model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,) , dtype = tf.string)  # shape = (1 , ) çünkü it is going to see one sequence at a time
x = text_vectorizer(inputs)  # Turn the input text into numbers
x = embedding(x) # Create an embedding of the numberized inputs.
x = layers.GlobalAveragePooling1D()(x) # Lower the dimensionality of the embedding. Eğer pooling layer eklemezsek her token için ayrı ayrı predict yapıyor. Pooling layer tek bir uzun vectore çeviriyor.
outputs = layers.Dense( 1 , activation = "sigmoid")(x)  # Create an output layer , We have binary outputs so we use 1 hidden unit and activation will be sigmoid

model_1 = tf.keras.Model(inputs , outputs , name = "model_1_dense")

In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [37]:
# Compile a model
model_1.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )

# Fit the model

hist_1 = model_1.fit(x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels)  ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_1_dense")])   # validation datayı tuple formatta yazmamız gerekiyor.

Saving TensorBoard log files to: model_logs/model_1_dense/20230818-142250
Epoch 1/5
215/215 [==============================] - 6s 20ms/step - loss: 0.6105 - accuracy: 0.6916 - val_loss: 0.5376 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.4402 - accuracy: 0.8180 - val_loss: 0.4795 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3462 - accuracy: 0.8634 - val_loss: 0.4575 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.2839 - accuracy: 0.8920 - val_loss: 0.4609 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.2364 - accuracy: 0.9121 - val_loss: 0.4861 - val_accuracy: 0.7940


In [38]:
# Check the results
model_1.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4861 - accuracy: 0.7940


[0.4860583245754242, 0.7939632534980774]

In [39]:
# Make some predictions and evaluate those
model_1_preds = model_1.predict(val_sentences)
model_1_preds.shape

24/24 [==============================] - 1s 6ms/step


(762, 1)

In [40]:
# Look at a single prediction
model_1_preds[0]

array([0.28718972], dtype=float32)

In [41]:
 val_labels  # Şu an val labels ile predlerimizi karşılaştıramıyoruz çünkü biz apple to apple gibi aynı şeyleri karşılaştırmamız gerek. Zaten binary classificationdan bildigimiz üzere şu an predictlerimiz probabilities şeklinde yani 0 la 1 arasında değerler alıyor. 0.5 in üstündeki değerleri 1 olarak alıcaz altındaki değerleri ise 0 olarak alıcaz. Bu şekilde pred değerlerimizle ground truth değerleri karşılaştırabiliriz.


array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [42]:
# Convert model prediction probabilities to label format

model_1_pred = tf.squeeze(tf.round(model_1_preds))
model_1_pred[ : 10] , model_1_pred.shape

(<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>,
 TensorShape([762]))

In [43]:
# Calculate our model_1 results

model_1_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_1_pred
                                    )
model_1_results

{'accuracy': 0.7939632545931758,
 'precision': 0.8025297569280951,
 'recall': 0.7939632545931758,
 'f1-score': 0.7900777653888648}

In [44]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

# Looks like our baseline is outperforming our first deep learning model

# Embedding layer en başta random değerler alıyor. Sonrasında her epochta internal weights ini uptade for each token.  

# Visualizing learned embeddings

In [45]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [46]:
# Get the weight matrix of embedding layer
# These are the numerical representation of each token in our training data
# Getlayer kullanırken summary içindeki ismi kullanıyoruz.
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [47]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


# Now we have got the embedding matrix our model has learned to represent our tokens , lets visualize this

Handy tool form tensoflow : Tensorflow Projector

https://www.tensorflow.org/text/guide/word_embeddings

In [48]:
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# To upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

https://www.youtube.com/watch?v=SEnXr6v2ifU


# Model 2: LSTM

LSTM = Long Short Term Memory

Our structure of an RNN typically looks like this:

'''
Input(text) -> Tokenize -> Embedding -> Layers (RNN's / dense) -> Output(label probability)
'''

In [50]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape = (1 ,)  , dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(64 , return_sequences = True)(x)  # When you are stacking RNN cells together , you need to set return_sequences = True. Yani eğer birden fazla lstm layer kullanacaksak son lstm layer hariç true atamamız lazım.
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64 , activation = "relu")(x)
outputs = layers.Dense(1 , activation = "sigmoid")(x)

model_2 = tf.keras.Model(inputs , outputs)


In [51]:
model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
___________________________________________________

In [52]:
# Compile a model

model_2.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]

                )

hist_2 = model_2.fit(x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels)  ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_2_dense")])

Saving TensorBoard log files to: model_logs/model_2_dense/20230818-142317
Epoch 1/5
215/215 [==============================] - 10s 22ms/step - loss: 0.2246 - accuracy: 0.9177 - val_loss: 0.6029 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1537 - accuracy: 0.9429 - val_loss: 0.5861 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 6s 29ms/step - loss: 0.1242 - accuracy: 0.9531 - val_loss: 0.6949 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 7s 32ms/step - loss: 0.1036 - accuracy: 0.9613 - val_loss: 0.7845 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0826 - accuracy: 0.9673 - val_loss: 0.9734 - val_accuracy: 0.7756


In [53]:
model_2.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.9734 - accuracy: 0.7756


[0.9734324216842651, 0.7755905389785767]

In [54]:
model_2_preds = model_2.predict(val_sentences)

24/24 [==============================] - 0s 4ms/step


In [55]:
model_2_pred = tf.squeeze(tf.round(model_2_preds))
model_2_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [56]:
model_2_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_2_pred
                                    )
model_2_results

{'accuracy': 0.7755905511811023,
 'precision': 0.7790436087799748,
 'recall': 0.7755905511811023,
 'f1-score': 0.7728084240126466}

# Model 3 : GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell(layers) has similar features to an LSTM cell but has less parameters

In [60]:
# Build an RNN using the GRU cell

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,) , dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = layers.GRU(64)(x)

# x = layers.LSTM( 64 , return_sequences= True)(x)

# x = layers.GRU(64)(x)  böyle de yapabiliriz göstermek icin sadece comment line a aldım.

# x = layers.Dense( 64 , activation = "relu")(x)

# x = layers.GlobalAveragePooling1D()(x)  return sequences true yaparsak global average pooling layer eklememiz gerek ama yapmazsak gerek yok.

outputs = layers.Dense(1 , activation = "sigmoid" , name = "output_layer")(x)

model_3 = tf.keras.Model(inputs , outputs)


In [61]:
model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________

In [62]:
# Compile a model

model_3.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )



In [63]:
# Fit the model
hist_3 = model_3.fit(x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels)  ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_3_dense")])

Saving TensorBoard log files to: model_logs/model_3_dense/20230818-143159
Epoch 1/5
215/215 [==============================] - 7s 21ms/step - loss: 0.1600 - accuracy: 0.9356 - val_loss: 0.6836 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0840 - accuracy: 0.9724 - val_loss: 0.8863 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0720 - accuracy: 0.9721 - val_loss: 0.9030 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0587 - accuracy: 0.9747 - val_loss: 1.0831 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0560 - accuracy: 0.9756 - val_loss: 1.1891 - val_accuracy: 0.7677


In [64]:
model_3.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 9ms/step - loss: 1.1891 - accuracy: 0.7677


[1.189102053642273, 0.7677165269851685]

In [67]:
model_3_preds_probs = model_3.predict(val_sentences)
model_3_preds_probs , model_3_preds_probs.shape

24/24 [==============================] - 0s 4ms/step


(array([[9.33567237e-04],
        [7.86224008e-01],
        [9.99835014e-01],
        [1.52615368e-01],
        [7.31422479e-05],
        [9.99648929e-01],
        [1.51633501e-01],
        [9.99940634e-01],
        [9.99859989e-01],
        [8.72723520e-01],
        [1.40119970e-04],
        [7.53337026e-01],
        [1.40699587e-04],
        [1.96753323e-01],
        [6.71073503e-05],
        [1.88254868e-03],
        [1.32227331e-04],
        [1.13866619e-04],
        [1.53231816e-02],
        [9.99298871e-01],
        [9.99843597e-01],
        [4.11053989e-05],
        [9.99459267e-01],
        [9.93008609e-04],
        [9.99783933e-01],
        [9.99913633e-01],
        [3.84999439e-04],
        [6.95713679e-04],
        [2.43780334e-04],
        [2.73990899e-01],
        [9.84578609e-01],
        [8.62300803e-04],
        [1.90395792e-03],
        [8.19994253e-04],
        [4.83461648e-01],
        [2.38137498e-01],
        [9.99788165e-01],
        [3.17075402e-01],
        [1.2

In [68]:
model_3_pred_probs = tf.squeeze(tf.round(model_3_preds_probs))
model_3_pred_probs

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [69]:
model_3_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_3_pred_probs
                                    )
model_3_results

{'accuracy': 0.7677165354330708,
 'precision': 0.7712255031085189,
 'recall': 0.7677165354330708,
 'f1-score': 0.7646846187166754}

# Model 4 : Bidirectional RNN

Normal RNN's go from left to right (just like you would read an English sencente) however , a bidirectional RNN goes from right to left as well as left to right.

In [83]:
# Create a model
inputs = layers.Input( shape = (1 ,) , dtype = tf.string)

x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional( layer = layers.LSTM(64 , return_sequences = True))(x)
x = layers.Bidirectional( layer = layers.GRU(64))(x)
outputs = layers.Dense( 1 , activation = "sigmoid" )(x)

model_4 = tf.keras.Model(inputs , outputs)



In [84]:
model_4.summary()  # hem left hem righttan baslayıp okudugumuz icin 64 iki katına cıkıyor. Cift taraflı process

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_9 (Bidirectio  (None, 15, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 128)              74496     
 onal)                                                           
                                                           

In [85]:
# Compile a model
model_4.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )



In [93]:
# Fit the model
hist_4 = model_4.fit(x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels)  ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_4_dense")])

Saving TensorBoard log files to: model_logs/model_4_dense/20230818-153804
Epoch 1/5
215/215 [==============================] - 11s 47ms/step - loss: 0.0441 - accuracy: 0.9780 - val_loss: 1.3637 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 9s 42ms/step - loss: 0.0367 - accuracy: 0.9816 - val_loss: 1.5140 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 8s 39ms/step - loss: 0.0314 - accuracy: 0.9837 - val_loss: 1.6644 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0305 - accuracy: 0.9834 - val_loss: 1.5602 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 9s 42ms/step - loss: 0.0314 - accuracy: 0.9841 - val_loss: 1.5137 - val_accuracy: 0.7638


In [87]:
model_4.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 8ms/step - loss: 1.4362 - accuracy: 0.7612


[1.4361608028411865, 0.76115483045578]

In [90]:
model_4_pred = model_4.predict(val_sentences)
model_4_pred

24/24 [==============================] - 0s 14ms/step


array([[5.23207884e-04],
       [6.66913211e-01],
       [9.99972939e-01],
       [2.18059972e-01],
       [3.93260452e-05],
       [9.99886870e-01],
       [4.62646335e-01],
       [9.99984443e-01],
       [9.99967933e-01],
       [9.56643939e-01],
       [5.86530914e-05],
       [9.06733274e-01],
       [6.06115718e-05],
       [3.43738496e-01],
       [5.27202064e-05],
       [4.50437976e-04],
       [7.10137829e-05],
       [5.94688427e-05],
       [3.72070819e-03],
       [9.99830544e-01],
       [9.98442471e-01],
       [4.33132191e-05],
       [9.99904156e-01],
       [1.72385568e-04],
       [9.99909341e-01],
       [9.99983668e-01],
       [1.55302798e-04],
       [4.81643772e-04],
       [7.30544853e-05],
       [6.89695358e-01],
       [9.99340534e-01],
       [3.46912304e-04],
       [2.05323263e-03],
       [3.98569711e-04],
       [5.00808582e-02],
       [3.65964979e-01],
       [9.99947011e-01],
       [8.83281156e-02],
       [4.25113831e-03],
       [9.99971926e-01],


In [91]:
model_4_preds = tf.squeeze(tf.round(model_4_pred))
model_4_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [92]:
model_4_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_4_preds
                                    )
model_4_results

{'accuracy': 0.7611548556430446,
 'precision': 0.7627664723727715,
 'recall': 0.7611548556430446,
 'f1-score': 0.7588524417705016}

# Model 5 : 1D Convolutional Neural Network

We have used CNN's for images but images are typically 2D (height x width) however , our text data is 1D.

Previously we used Conv2D for our image data but we are going to use Conv1D for our text data.

The typical structure of a Conv1D model for sequences (in our case , text):

Inputs(Text) -> Tokenization -> Embedding -> Layers (Conv1D + pooling) -> Outputs (class probabilities)


In [100]:
# Test out our embedding layer , Conv1D layer and max pooling

embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D( filters = 32 ,
                        kernel_size = 5 ,  # It looks at 5 words at a time (also referred as ngram because aynı anda 5 kelimeye bakıyor.)
                         activation =   "relu" ,
                         padding = "valid"   # Bu bizim conv_1d_output umuzun shapeinin 11 çıkmasına sebep oluyor. padding = same deseydik yine 15 olacaktı token outputumuz. valid atadıgımızda the output is smaller than the input shape.
                         )  # padding valid atamazsak some of the tokens at the end of our sequences may get missed. Valid bunu engelliyor.

conv_1d_output = conv_1d(embedding_test)  # Past test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)  # Equivalent to get the most important features or get the feature with highest value
embedding_test.shape , conv_1d_output.shape , max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [105]:
# Create a model

inputs = layers.Input( shape = ( 1 ,) , dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = layers.Conv1D( 64 , 5 , activation = "relu")(x)

x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense( 1 , activation = "sigmoid")(x)

model_5 = tf.keras.Model(inputs , outputs)



In [106]:
model_5.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_9 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_12 (Dense)            (None, 1)                 65 

In [107]:
# Compile a model

model_5.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )

In [109]:
hist_5 = model_5.fit( x = train_sentences ,
                      y = train_labels ,
                      epochs = 5 ,
                      validation_data = (val_sentences , val_labels) ,
                      callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_5_dense")]
                      )

Saving TensorBoard log files to: model_logs/model_5_dense/20230818-170416
Epoch 1/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0431 - accuracy: 0.9807 - val_loss: 1.3741 - val_accuracy: 0.7493
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0421 - accuracy: 0.9809 - val_loss: 1.3229 - val_accuracy: 0.7428
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0393 - accuracy: 0.9826 - val_loss: 1.3256 - val_accuracy: 0.7454
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0397 - accuracy: 0.9809 - val_loss: 1.3269 - val_accuracy: 0.7454
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0386 - accuracy: 0.9813 - val_loss: 1.3076 - val_accuracy: 0.7480


In [110]:
model_5_preds = model_5.predict(val_sentences)
model_5_preds

24/24 [==============================] - 0s 3ms/step


array([[2.32050598e-01],
       [8.39550614e-01],
       [9.99944866e-01],
       [4.62954715e-02],
       [6.16197440e-07],
       [9.80061114e-01],
       [9.85522389e-01],
       [9.99966085e-01],
       [9.99998331e-01],
       [8.60878229e-01],
       [3.16967544e-07],
       [9.57833588e-01],
       [6.28220550e-06],
       [8.38966891e-02],
       [3.82026919e-05],
       [8.16079415e-03],
       [3.29619972e-04],
       [3.00956635e-05],
       [2.38605756e-02],
       [9.92576838e-01],
       [5.30821145e-01],
       [2.75959788e-08],
       [9.99765635e-01],
       [6.22869193e-05],
       [9.99998629e-01],
       [1.00000000e+00],
       [1.36772333e-05],
       [9.84845590e-03],
       [1.03043008e-03],
       [3.40016782e-01],
       [9.54839349e-01],
       [1.50516089e-05],
       [4.63945508e-01],
       [6.50518283e-04],
       [3.50803398e-02],
       [8.11906397e-01],
       [9.99994636e-01],
       [3.26676875e-01],
       [3.84348212e-03],
       [9.99958098e-01],


In [112]:
model_5_pred = tf.squeeze(tf.round(model_5_preds))
model_5_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [114]:
model_5_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_5_pred
                                    )
model_5_results

{'accuracy': 0.7480314960629921,
 'precision': 0.7480600429524334,
 'recall': 0.7480314960629921,
 'f1-score': 0.7465302222155744}

# Model 6 : TensorFlow hub pre-trained sentence encoder

Lets try and use transfer learning for NLP , universal sentence encoder from tensorflow hub

In [116]:
sample_sentence

"There's a flood in my street!"

In [115]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embed_samples = embed([sample_sentence , "When you can the universal sentence encoder on a sentence , it turns it into numbers"])
print(embed_samples[0][ : 50])

tf.Tensor(
[-0.01157025  0.02485911  0.02878051 -0.012715    0.03971541  0.08827761
  0.02680988  0.05589838 -0.01068731 -0.00597293  0.00639321 -0.01819516
  0.00030816  0.09105889  0.05874645 -0.03180629  0.01512474 -0.05162925
  0.00991366 -0.06865345 -0.04209306  0.0267898   0.03011009  0.00321065
 -0.00337968 -0.04787356  0.0226672  -0.00985927 -0.04063615 -0.01292093
 -0.04666382  0.05630299 -0.03949255  0.00517682  0.02495827 -0.07014439
  0.0287151   0.0494768  -0.00633978 -0.08960193  0.02807119 -0.00808364
 -0.01360601  0.05998649 -0.10361788 -0.05195372  0.00232958 -0.02332531
 -0.03758106  0.03327729], shape=(50,), dtype=float32)


In [122]:
embed_samples[0].shape

TensorShape([512])

In [123]:
# Create a Keras layer using the USE pre-trained layer from tensorflow hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4" ,
                                        input_shape = [] , # We defined input shape as empty vector because the input to this layer can be a variable length. This pre-trained layer can take any variable length. Any variable length can go into this module but it will always output a 512 feature vector no matter what length sequence goes in. The output will always be 512.
                                        dtype = tf.string ,
                                        trainable = False ,
                                        name = "USE"
                                        )
# Text vectorization layera ihtiyacımız yok cünkü bu modelin icinde zaten text vectorization yapıyor kendisi.

In [124]:
# Create a model with using Sequential API

model_6 = tf.keras.Sequential(
    [
        sentence_encoder_layer ,  # Bu modelde input layerımız direkt bu oluyor.
        #layers.Dense(64 , activation = "relu")  yani istersek böyle lane stackleyebiliyoruz.
        layers.Dense( 1 , activation = "sigmoid")


    ] , name = "model_6_USE"

)

In [125]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_13 (Dense)            (None, 1)                 513       
                                                                 
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [126]:
# Compile a model
model_6.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )

In [127]:
# Fit the model
hist_6 = model_6.fit( x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels) ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_6_dense")]

                      )

Saving TensorBoard log files to: model_logs/model_6_dense/20230818-181802
Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.6499 - accuracy: 0.7227 - val_loss: 0.6127 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.5829 - accuracy: 0.7835 - val_loss: 0.5629 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.5399 - accuracy: 0.7942 - val_loss: 0.5311 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.5111 - accuracy: 0.7977 - val_loss: 0.5099 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4909 - accuracy: 0.7996 - val_loss: 0.4957 - val_accuracy: 0.7861


In [128]:
model_6_preds = model_6.predict(val_sentences)
model_6_preds


24/24 [==============================] - 0s 8ms/step


array([[0.36401966],
       [0.68358135],
       [0.85293466],
       [0.35171145],
       [0.6507671 ],
       [0.7427859 ],
       [0.8260346 ],
       [0.8430263 ],
       [0.7469515 ],
       [0.19322678],
       [0.5210855 ],
       [0.48201615],
       [0.40863255],
       [0.4603998 ],
       [0.4979054 ],
       [0.18951929],
       [0.36463842],
       [0.4836935 ],
       [0.5091783 ],
       [0.5437844 ],
       [0.635258  ],
       [0.31116167],
       [0.48851126],
       [0.16743603],
       [0.6126479 ],
       [0.79695624],
       [0.2315622 ],
       [0.34017333],
       [0.25505143],
       [0.36914977],
       [0.5562811 ],
       [0.7497512 ],
       [0.4490748 ],
       [0.4923389 ],
       [0.53538215],
       [0.21106592],
       [0.78324765],
       [0.16556041],
       [0.19536407],
       [0.8122299 ],
       [0.23541427],
       [0.32110667],
       [0.55149484],
       [0.55582094],
       [0.28161234],
       [0.7409025 ],
       [0.31845817],
       [0.830

In [129]:
model_6_pred = tf.squeeze(tf.round(model_6_preds))
model_6_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [130]:
model_6_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_6_pred
                                    )
model_6_results

{'accuracy': 0.7860892388451444,
 'precision': 0.7863431959164349,
 'recall': 0.7860892388451444,
 'f1-score': 0.7850582651599072}

In [131]:
# Try to improve our model_6

model_6_improve = tf.keras.Sequential(
    [
        sentence_encoder_layer ,
        layers.Dense( 64 , activation =  "relu") ,
        layers.Dense( 1 , activation = "sigmoid" , name = "output_layer")

    ]
)

In [132]:
# Compile the model
model_6_improve.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )

In [133]:
# Fit the model
hist_6_improve = model_6_improve.fit( x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels) ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_6_improve_dense")]

                      )

Saving TensorBoard log files to: model_logs/model_6_improve_dense/20230818-183008
Epoch 1/5
215/215 [==============================] - 4s 11ms/step - loss: 0.5028 - accuracy: 0.7854 - val_loss: 0.4480 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4141 - accuracy: 0.8146 - val_loss: 0.4375 - val_accuracy: 0.8071
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3996 - accuracy: 0.8208 - val_loss: 0.4339 - val_accuracy: 0.8123
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3927 - accuracy: 0.8241 - val_loss: 0.4301 - val_accuracy: 0.8097
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3859 - accuracy: 0.8304 - val_loss: 0.4275 - val_accuracy: 0.8084


In [134]:
model_6_improve_preds = model_6_improve.predict(val_sentences)
model_6_improve_preds


24/24 [==============================] - 0s 8ms/step


array([[0.16276628],
       [0.79003876],
       [0.98984116],
       [0.21169765],
       [0.6732754 ],
       [0.75170535],
       [0.97835785],
       [0.9790968 ],
       [0.9541548 ],
       [0.09700762],
       [0.6214952 ],
       [0.44272122],
       [0.16536315],
       [0.53145486],
       [0.24104969],
       [0.0315002 ],
       [0.32123494],
       [0.59869957],
       [0.3495958 ],
       [0.3073196 ],
       [0.9147935 ],
       [0.10915114],
       [0.36180648],
       [0.03740163],
       [0.8765997 ],
       [0.97064894],
       [0.06189346],
       [0.12721719],
       [0.10696267],
       [0.28704932],
       [0.48814392],
       [0.91863847],
       [0.329686  ],
       [0.28129384],
       [0.44719774],
       [0.08466875],
       [0.9821984 ],
       [0.06073403],
       [0.03722097],
       [0.98584557],
       [0.08391458],
       [0.25869554],
       [0.49620867],
       [0.53998363],
       [0.26848435],
       [0.96621764],
       [0.3619913 ],
       [0.953

In [135]:
model_6_improve_pred = tf.squeeze(tf.round(model_6_improve_preds))
model_6_improve_pred


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [136]:
model_6_improve_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_6_improve_pred
                                    )
model_6_improve_results

{'accuracy': 0.8083989501312336,
 'precision': 0.8106890532903106,
 'recall': 0.8083989501312336,
 'f1-score': 0.8067678554730104}